In [2]:
import numpy as np

def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def numerical_derivative(f, x):
    delta_x = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x)
        x[idx] = float(tmp_val) - delta_x
        fx2 = f(x)
        grad[idx] = (fx1 - fx2) / (2 * delta_x)
        x[idx] = tmp_val
        it.iternext()
    return grad

class LogicGate:
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name
        self.xdata = xdata
        self.tdata = tdata.reshape(-1, 1)
        self.W = np.random.rand(xdata.shape[1], 1)
        self.b = np.random.rand(1)
        self.learning_rate = 1e-2

    def loss_func(self):
        delta = 1e-7
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
        return -np.sum(self.tdata * np.log(y + delta) + (1 - self.tdata) * np.log(1 - y + delta))

    def train(self):
        f = lambda x: self.loss_func()
        for step in range(180001):
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
            if step % 10000 == 0:
                print(f"{self.name} step {step}, loss = {self.loss_func()}")

    def predict(self, x):
        z = np.dot(x, self.W) + self.b
        y = sigmoid(z)
        logical_val = 1 if y > 0.5 else 0
        return y, logical_val

    def accuracy(self, test_input, test_target):
        matched = 0
        for idx in range(len(test_input)):
            (_, logical_val) = self.predict(test_input[idx])
            if logical_val == test_target[idx]:
                matched += 1
        return matched / len(test_input)



In [3]:
xdata = np.array([[0,0], [0,1], [1,0], [1,1]])

In [4]:
or_tdata = np.array([0,1,1,1])
OR_gate = LogicGate("OR_GATE", xdata, or_tdata)
OR_gate.train()

OR_GATE step 0, loss = 1.9148486850851236
OR_GATE step 10000, loss = 0.09452893203252861
OR_GATE step 20000, loss = 0.04671747577421242
OR_GATE step 30000, loss = 0.030912491817671462
OR_GATE step 40000, loss = 0.02307216299267471
OR_GATE step 50000, loss = 0.01839521086281991
OR_GATE step 60000, loss = 0.015290832175252017
OR_GATE step 70000, loss = 0.01308094159244645
OR_GATE step 80000, loss = 0.01142803052486527
OR_GATE step 90000, loss = 0.010145302774826614
OR_GATE step 100000, loss = 0.009121038078144756
OR_GATE step 110000, loss = 0.008284335453805119
OR_GATE step 120000, loss = 0.00758803892291547
OR_GATE step 130000, loss = 0.006999569216160975
OR_GATE step 140000, loss = 0.006495697648163727
OR_GATE step 150000, loss = 0.006059417324662433
OR_GATE step 160000, loss = 0.005677991497321398
OR_GATE step 170000, loss = 0.005341692978027845
OR_GATE step 180000, loss = 0.005042965102846402


In [20]:
test_xdata= np.array([[0,0], [0,1], [1,0], [1,1]])
test_tdata = np.array([0,1,1,1])
accuracy_ret = OR_gate.accuracy(test_xdata, test_tdata)
print(accuracy_ret)

1.0


In [21]:
nand_tdata = np.array([1,1,1,0])
NAND_gate = LogicGate("NAND_GATE", xdata, nand_tdata)
NAND_gate.train()

NAND_GATE step 0, loss = 2.89433114382009
NAND_GATE step 10000, loss = 0.1730549685692322
NAND_GATE step 20000, loss = 0.0874115769582245
NAND_GATE step 30000, loss = 0.05817934313335662
NAND_GATE step 40000, loss = 0.04352697559698127
NAND_GATE step 50000, loss = 0.034744455669909274
NAND_GATE step 60000, loss = 0.028899641952677677
NAND_GATE step 70000, loss = 0.02473233028429601
NAND_GATE step 80000, loss = 0.021612149007285412
NAND_GATE step 90000, loss = 0.019189079530537285
NAND_GATE step 100000, loss = 0.0172533080321512
NAND_GATE step 110000, loss = 0.015671463162308663
NAND_GATE step 120000, loss = 0.014354732092298698
NAND_GATE step 130000, loss = 0.013241698365042768
NAND_GATE step 140000, loss = 0.012288540835224838
NAND_GATE step 150000, loss = 0.01146315732629979
NAND_GATE step 160000, loss = 0.010741494836069024
NAND_GATE step 170000, loss = 0.010105176914078525
NAND_GATE step 180000, loss = 0.009539923217027171


In [22]:
test_xdata = np.array([[0,0], [0,1], [1,0], [1,1]])
test_tdata = np.array([1,1,1,0])
accuracy_ret = NAND_gate.accuracy(test_xdata, test_tdata)
print(accuracy_ret)

1.0


In [23]:
hidden_output = []
for input_data in xdata:
    y, or_val = OR_gate.predict(input_data)
    y, nand_val = NAND_gate.predict(input_data)
    hidden_output.append([or_val, nand_val])
hidden_output = np.array(hidden_output)
hidden_output

array([[0, 1],
       [1, 1],
       [1, 1],
       [1, 0]])

In [24]:
and_tdata = np.array([0,1,1,0]) 
AND_gate = LogicGate("AND_GATE", hidden_output, and_tdata)
AND_gate.train()

AND_GATE step 0, loss = 3.028425182651552
AND_GATE step 10000, loss = 0.18095813186097387
AND_GATE step 20000, loss = 0.08907252483967518
AND_GATE step 30000, loss = 0.05881140413893336
AND_GATE step 40000, loss = 0.043837903627709895
AND_GATE step 50000, loss = 0.03492096236666671
AND_GATE step 60000, loss = 0.029009397182847528
AND_GATE step 70000, loss = 0.02480499824721628
AND_GATE step 80000, loss = 0.021662505058688143
AND_GATE step 90000, loss = 0.019225192394712383
AND_GATE step 100000, loss = 0.017279901454876127
AND_GATE step 110000, loss = 0.015691457495103533
AND_GATE step 120000, loss = 0.014370013719559333
AND_GATE step 130000, loss = 0.013253530523277718
AND_GATE step 140000, loss = 0.012297795368805224
AND_GATE step 150000, loss = 0.011470451554680915
AND_GATE step 160000, loss = 0.010747275575296816
AND_GATE step 170000, loss = 0.010109773798142643
AND_GATE step 180000, loss = 0.00954358352446667


In [10]:
for input_data in xdata:
    _, or_val = OR_gate.predict(input_data)
    _, nand_val = NAND_gate.predict(input_data)
    and_input = np.array([or_val, nand_val])
    _, xor_val = AND_gate.predict(and_input)
    print(f"{input_data} => {xor_val}")

xor_test_input = hidden_output
xor_test_target = and_tdata
accuracy = AND_gate.accuracy(xor_test_input, xor_test_target)
print("XOR Accuracy =>", accuracy)

[0 0] => 0
[0 1] => 1
[1 0] => 1
[1 1] => 0
XOR Accuracy => 1.0
